In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import calendar
import datetime as dt
import datetime
import pickle
import azureml.train.automl
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from math import sqrt
from lifelines.utils import k_fold_cross_validation
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import matplotlib.pyplot as plt
from matplotlib.image import imread
import pandas as pd
import seaborn as sns
from sklearn.datasets.samples_generator import (make_blobs,
                                                make_circles,
                                                make_moons)
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score

%matplotlib inline
sns.set_context('notebook')
plt.style.use('fivethirtyeight')
from warnings import filterwarnings
filterwarnings('ignore')

Duplicate key in file '/Users/at185217/.matplotlib/matplotlibrc' line #2.


In [3]:
df = pd.read_csv('EDL_Train.csv')
df.columns = df.columns.str.replace(r'train_data_t2p.', '')
df['customer_nbr'] = df['customer_nbr'].astype(str)
df['invoice_nbr'] = df['invoice_nbr'].astype(str)
df["ratio_closed_0"] = df["closed_bucket_0"]/df["count_paid_invoices"]
df["ratio_closed_1"] = df["closed_bucket_1"]/df["count_paid_invoices"]
df["ratio_closed_2"] = df["closed_bucket_2"]/df["count_paid_invoices"]
df["ratio_closed_3"] = df["closed_bucket_3"]/df["count_paid_invoices"]
df["ratio_closed_4"] = df["closed_bucket_4"]/df["count_paid_invoices"]
df = df.loc[df['customer_segment'] == 1]
print(df.shape)
print(df.customer_segment.value_counts())

(808209, 100)
1    808209
Name: customer_segment, dtype: int64


In [4]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'])
df['actual_date_closed'].replace('4712-12-31','2020-12-31', inplace = True)
df['actual_date_closed'] = pd.to_datetime(df['actual_date_closed'])

test = df.loc[(df['actual_date_closed'] >= '2020-01-01') & (df['invoice_date'] < '2020-01-01')]

In [5]:
test.head()

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4
2883505,10000260,1,6501309879,53.300,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,23,6,0.260,1488.200,585.400,0.390,1.500,0.000,0,0.000,0.000,0.000,0.000,0.000,17,6,0,0,0,0,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.739,0.261,0.000,0.000,0.000
2883506,10001749,1,200370484,130.000,30,1,0,4,188,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-06-27,2019-06-27,2019-07-27,2020-12-31,27,6,3.000,27,7,5.000,27,6,3.000,0,1,0,0,6,0.190,32,26,0.810,7478.460,5944.250,0.790,133.080,19.000,19,1.000,5611.670,5611.670,1.000,221.260,6,5,1,0,20,19,52,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.188,0.156,0.031,0.000,0.625
2883507,10001902,1,6501314548,113.000,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,13,3,0.230,1694.000,339.000,0.200,10.000,8.000,8,1.000,904.000,904.000,1.000,154.750,10,3,0,0,0,7,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.769,0.231,0.000,0.000,0.000
2883508,10002353,1,200371162,87.000,30,1,0,4,188,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-06-27,2019-06-27,2019-07-27,2020-12-31,27,6,3.000,27,7,5.000,27,6,3.000,0,1,0,0,0,0.000,4,4,1.000,1935.000,1935.000,1.000,150.750,8.000,8,1.000,709.540,709.540,1.000,113.880,0,1,1,0,2,6,32,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.000,0.250,0.250,0.000,0.500
2883509,10002353,1,200387119,90.020,30,1,0,4,161,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-07-24,2019-07-24,2019-08-23,2020-12-31,24,7,2.000,23,8,4.000,24,7,2.000,0,0,0,0,0,0.000,4,4,1.000,1935.000,1935.000,1.000,150.750,8.000,8,1.000,709.540,709.540,1.000,113.880,0,1,1,0,2,6,32,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.000,0.250,0.250,0.000,0.500


In [6]:
test.actual_date_closed.value_counts()

2020-12-31    111895
2020-01-09     16161
2020-01-14      6046
2020-02-07      4761
2020-01-22      2053
2020-01-13      1871
2020-01-16      1856
2020-01-10      1809
2020-01-23      1729
2020-02-05      1720
2020-01-24      1304
2020-01-28      1281
2020-01-30      1255
2020-02-06      1011
2020-01-27      1009
2020-01-31       998
2020-02-11       981
2020-01-29       836
2020-01-07       825
2020-01-17       755
2020-02-10       667
2020-01-20       613
2020-01-21       606
2020-01-06       549
2020-01-03       546
2020-01-04       511
2020-01-01       377
2020-01-08       318
2020-02-04       267
2020-02-03       248
2020-01-15       190
2020-01-02       136
2020-01-11        97
2020-01-25        74
2020-01-18        40
2020-02-01        39
2020-02-12        33
2020-01-26        16
2020-02-08        13
2020-12-05         1
2020-01-12         1
Name: actual_date_closed, dtype: int64

In [7]:
test.shape

(165498, 100)

##### Uploading trained model

In [8]:
# Load Model
filename = 'Survival_Model_EDL_Data.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [9]:
test2 = test[['country_code','invoice_base_amount','commited_days', 'invoice_category', 'dso2', 'financial_banking_cfi',
            'financial_banking_major', 'financial_channel', 'financial_other_financial_serv','hospitality_channel',
            'hospitality_eating_drinking_qs','hospitality_eating_drinking_ts','hospitality_sports_recreation','retail_channel',
            'retail_cstore_and_petroleum','retail_distribution_wholesale','retail_drug','retail_food_and_mass',
            'retail_public_sector','retail_specialty_retail','t_and_t_technology','tlg_channel',
            'tlg_gaming','tlg_lodging','tlg_travel',
            'month_num_created','day_of_week_created','day_of_month_due','month_num_due','day_of_week_due',
            'ratio_late_paid_amount',
            'ratio_closed_0', 'ratio_closed_1', 'ratio_closed_2', 'ratio_closed_3', 'ratio_closed_4', 
            'hw','sw','swm','hwm','ps','ts','other_category','paid_status']]

In [10]:
test3=test2.fillna(0)

In [11]:
#Predicting from the trained model
y_pred=loaded_model.predict_median(test3)

In [12]:
test['Pred_T2P'] = y_pred

In [13]:
test.head()

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P
2883505,10000260,1,6501309879,53.300,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,23,6,0.260,1488.200,585.400,0.390,1.500,0.000,0,0.000,0.000,0.000,0.000,0.000,17,6,0,0,0,0,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.739,0.261,0.000,0.000,0.000,30.000
2883506,10001749,1,200370484,130.000,30,1,0,4,188,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-06-27,2019-06-27,2019-07-27,2020-12-31,27,6,3.000,27,7,5.000,27,6,3.000,0,1,0,0,6,0.190,32,26,0.810,7478.460,5944.250,0.790,133.080,19.000,19,1.000,5611.670,5611.670,1.000,221.260,6,5,1,0,20,19,52,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.188,0.156,0.031,0.000,0.625,315.000
2883507,10001902,1,6501314548,113.000,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,13,3,0.230,1694.000,339.000,0.200,10.000,8.000,8,1.000,904.000,904.000,1.000,154.750,10,3,0,0,0,7,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.769,0.231,0.000,0.000,0.000,30.000
2883508,10002353,1,200371162,87.000,30,1,0,4,188,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-06-27,2019-06-27,2019-07-27,2020-12-31,27,6,3.000,27,7,5.000,27,6,3.000,0,1,0,0,0,0.000,4,4,1.000,1935.000,1935.000,1.000,150.750,8.000,8,1.000,709.540,709.540,1.000,113.880,0,1,1,0,2,6,32,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.000,0.250,0.250,0.000,0.500,366.000
2883509,10002353,1,200387119,90.020,30,1,0,4,161,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-07-24,2019-07-24,2019-08-23,2020-12-31,24,7,2.000,23,8,4.000,24,7,2.000,0,0,0,0,0,0.000,4,4,1.000,1935.000,1935.000,1.000,150.750,8.000,8,1.000,709.540,709.540,1.000,113.880,0,1,1,0,2,6,32,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.000,0.250,0.250,0.000,0.500,333.000


In [14]:
output_test=test.loc[~test['Pred_T2P'].isin([np.inf, -np.inf])]
output_test['Predicted_Date'] = pd.to_datetime(output_test['invoice_date']) + pd.to_timedelta(output_test['Pred_T2P'], unit='d')
output_test2=output_test.loc[(output_test['Predicted_Date']>='2020-01-01') & (output_test['Predicted_Date']<='2020-01-31')]
Pred = output_test2.invoice_base_amount.sum()
print(output_test2.invoice_base_amount.sum())

55344455.79


In [91]:
# Lets see what actually happened

In [15]:
actuals = df.loc[(df['actual_date_closed'] >= '2020-01-01') & (df['actual_date_closed'] <= '2020-01-31')]
actuals = actuals.dropna()
print(actuals.invoice_base_amount.sum())

65602583.13999999


In [93]:
actuals.actual_date_closed.describe()

count                   43848
unique                     29
top       2020-01-09 00:00:00
freq                    16160
first     2020-01-01 00:00:00
last      2020-01-31 00:00:00
Name: actual_date_closed, dtype: object

In [94]:
chk = output_test.loc[output_test['actual_date_closed'] < '2020-12-05' ]

In [95]:
chk.actual_date_closed.describe()

count                   53531
unique                     39
top       2020-01-09 00:00:00
freq                    16158
first     2020-01-01 00:00:00
last      2020-02-12 00:00:00
Name: actual_date_closed, dtype: object

In [96]:
chk.shape

(53531, 102)

In [97]:
chk.head()

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date
2883505,10000260,1,6501309879,53.300,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,23,6,0.260,1488.200,585.400,0.390,1.500,0.000,0,0.000,0.000,0.000,0.000,0.000,17,6,0,0,0,0,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.739,0.261,0.000,0.000,0.000,30.000,2020-01-09
2883507,10001902,1,6501314548,113.000,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,13,3,0.230,1694.000,339.000,0.200,10.000,8.000,8,1.000,904.000,904.000,1.000,154.750,10,3,0,0,0,7,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.769,0.231,0.000,0.000,0.000,30.000,2020-01-09
2883516,10002421,1,1102526067,706.000,30,1,0,0,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2019-12-28,2019-12-28,2020-01-27,2020-01-27,28,12,5.000,27,1,0.000,28,12,5.000,0,1,0,0,3,0.040,77,74,0.960,311173.450,309729.450,1.000,206.700,119.000,119,1.000,117461.810,117461.810,1.000,167.180,3,20,0,1,53,102,4,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.039,0.260,0.000,0.013,0.688,466.000,2021-04-07
2883523,10005470,1,6501294980,145.360,30,1,0,1,26,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-06,2019-12-06,2020-01-05,2020-01-09,6,12,4.000,5,1,6.000,6,12,4.000,0,0,1,1,0,0.000,21,7,0.330,3382.910,784.750,0.230,1.860,0.000,0,0.000,0.000,0.000,0.000,0.000,14,7,0,0,0,0,8,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.667,0.333,0.000,0.000,0.000,31.000,2020-01-06
2883524,10006090,1,6501288454,238.860,30,1,0,1,26,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-06,2019-12-06,2020-01-05,2020-01-09,6,12,4.000,5,1,6.000,6,12,4.000,0,0,1,1,0,0.000,158,59,0.370,37856.810,14221.140,0.380,3.240,1.000,1,1.000,238.860,238.860,1.000,111.000,99,59,0,0,0,1,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.627,0.373,0.000,0.000,0.000,31.000,2020-01-06


In [98]:
chk['Actual_T2P'] = (chk['actual_date_closed'] - chk['invoice_date']).dt.days

In [99]:
chk.head(20)

,customer_nbr,country_code,invoice_nbr,invoice_base_amount,commited_days,payment_term,invoice_category,delay_status,dso2,invoice_oks,tm_invoice,om_invoice,hsr_sca_invoice,sca_invoice,saas_oks_invoice,hsr_tm_invoice,hsr_invoice_oks,hsr_om_invoice,other_invoice_type_name,finance,retail,hospitality,t_and_t,tlg,financial_banking_cfi,financial_banking_major,financial_channel,financial_other_financial_serv,hospitality_channel,hospitality_eating_drinking_qs,hospitality_eating_drinking_ts,hospitality_sports_recreation,retail_channel,retail_cstore_and_petroleum,retail_distribution_wholesale,retail_drug,retail_food_and_mass,retail_public_sector,retail_specialty_retail,t_and_t_technology,t_and_t_telecom,tlg_channel,tlg_gaming,tlg_lodging,tlg_travel,other_industry,invoice_date,aging_date,payment_due_date,actual_date_closed,day_of_month_created,month_num_created,day_of_week_created,day_of_month_due,month_num_due,day_of_week_due,day_of_month_age,month_num_age,day_of_week_age,me_due_date,me_inv_date,mme_due_date,mme_inv_date,count_prepaid_invoices,ratio_prepaid_invoices,count_paid_invoices,count_delayed_invoices,ratio_delayed_invoices,paid_invoices_amount_us,delayed_paid_invoices_amount,ratio_late_paid_amount,avg_days_late_paid_invoices_closed,count_outstanding_invoices,count_delayed_outstanding_invoices,ratio_outstanding_invoices,outstanding_invoices_amount,delayed_outstanding_invoices_amount,ratio_late_outstanding_amount,avg_days_late_outstanding_invoices_open,closed_bucket_0,closed_bucket_1,closed_bucket_2,closed_bucket_3,closed_bucket_4,open_60_more,total_invoice_lines,hw,sw,swm,hwm,ps,ts,other_category,paid_status,customer_segment,ratio_closed_0,ratio_closed_1,ratio_closed_2,ratio_closed_3,ratio_closed_4,Pred_T2P,Predicted_Date,Actual_T2P
2883505,10000260,1,6501309879,53.300,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,23,6,0.260,1488.200,585.400,0.390,1.500,0.000,0,0.000,0.000,0.000,0.000,0.000,17,6,0,0,0,0,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.739,0.261,0.000,0.000,0.000,30.000,2020-01-09,30
2883507,10001902,1,6501314548,113.000,30,1,0,0,22,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-10,2019-12-10,2020-01-09,2020-01-09,10,12,1.000,9,1,3.000,10,12,1.000,0,0,1,1,0,0.000,13,3,0.230,1694.000,339.000,0.200,10.000,8.000,8,1.000,904.000,904.000,1.000,154.750,10,3,0,0,0,7,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.769,0.231,0.000,0.000,0.000,30.000,2020-01-09,30
2883516,10002421,1,1102526067,706.000,30,1,0,0,4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2019-12-28,2019-12-28,2020-01-27,2020-01-27,28,12,5.000,27,1,0.000,28,12,5.000,0,1,0,0,3,0.040,77,74,0.960,311173.450,309729.450,1.000,206.700,119.000,119,1.000,117461.810,117461.810,1.000,167.180,3,20,0,1,53,102,4,0.000,0.000,0.000,0.250,0.000,0.000,0.750,0,1,0.039,0.260,0.000,0.013,0.688,466.000,2021-04-07,30
2883523,10005470,1,6501294980,145.360,30,1,0,1,26,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-06,2019-12-06,2020-01-05,2020-01-09,6,12,4.000,5,1,6.000,6,12,4.000,0,0,1,1,0,0.000,21,7,0.330,3382.910,784.750,0.230,1.860,0.000,0,0.000,0.000,0.000,0.000,0.000,14,7,0,0,0,0,8,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.667,0.333,0.000,0.000,0.000,31.000,2020-01-06,34
2883524,10006090,1,6501288454,238.860,30,1,0,1,26,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-06,2019-12-06,2020-01-05,2020-01-09,6,12,4.000,5,1,6.000,6,12,4.000,0,0,1,1,0,0.000,158,59,0.370,37856.810,14221.140,0.380,3.240,1.000,1,1.000,238.860,238.860,1.000,111.000,99,59,0,0,0,1,4,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0,1,0.627,0.373,0.000,0.000,0.000,31.000,2020-01-06,34
2883526,10006195,1,6501291320,147.340,30,1,0,1,26,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-06,2019-12-06,2020-01-05,20

In [100]:
chk.dropna(inplace=True)
sqrt(mean_squared_error(y_true = chk['Actual_T2P'], y_pred = chk['Pred_T2P']))

98.32396693244841

In [101]:
mean_absolute_error(y_true = chk['Actual_T2P'], y_pred = chk['Pred_T2P'])

43.000467019110424

In [102]:
median_absolute_error(y_true = chk['Actual_T2P'], y_pred = chk['Pred_T2P'])

11.0

In [19]:
t = df.loc[(df['actual_date_closed'] >= '2020-01-01')]

In [20]:
t.invoice_base_amount.sum()

285023546.18

In [21]:
df.customer_nbr.nunique()

34675